In [1]:
# import dependencies
import RMF
import pickle

In [2]:
# set parameters

# The input RMF file.
# input_rmf_path = "/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_200uM_10ns/"
# input_rmf_path = "/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_0uM_10ns/1/100000.movie.rmf"

# reference output pb file from which info e.g. fg nup types can be extracted
ref_output_path = "/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_0uM_10ns/1/100000.pb"

In [3]:
def has_depth_with_site(root, i):
    """ returns true if node subtree thru first child is at least i
        levels, including the root node itself, and the lead is a site """
#  print root, i, len(root.get_children())
    if (i==1) and root.get_name()=="site":
        return True
    c = root.get_children()
    if len(c) == 0:
        return False
    return has_depth_with_site(c[0], i-1)

def _add_nodes(node, tf, type_prefixes, depth=0):
    '''
    node - rmf node to scan
    tf - typed factory
    type_prefixes - list of full type prefixes (e.g. "Nup1" for "Nup1N")

    adds only nodes whose type name begins with any of the specified type prefixes
    '''
    children = node.get_children()
    ret = []
    #print "inspecting", node.get_name()
    if len(children)==0:
        return ret
    if has_depth_with_site(node, 3) and tf.get_is(children[0]):
        child_type = tf.get(children[0]).get_type_name()
        if any([child_type.startswith(tp) for tp in type_prefixes]):
            ret.append(children)
    for c in children:
        ret += _add_nodes(c, tf,  type_prefixes, depth+1)
    return ret


def load_data(input_rmf_path, min_z_angstrom):
    x_coords = []
    y_coords = []
    z_coords = []

    for sim_i in range(10, 60):
        for rmf_i in range(50000, 150000, 50000):
            in_fh = RMF.open_rmf_file_read_only(f"{input_rmf_path}/{sim_i}/{rmf_i}.pb.final.rmf")
            rff = RMF.ReferenceFrameFactory(in_fh)
            tf = RMF.TypedFactory(in_fh)
            fg_types = [f"fg{x}" for x in range(32)]

            # load data
            type2chains={}
            for i, fg_type in enumerate(fg_types):
                type2chains[fg_type] = _add_nodes(in_fh.get_root_node(), tf, [fg_type])
                
            # set frame
            frames = in_fh.get_frames()
            in_fh.set_current_frame(frames[0])

            # read data
            for fg_i in range(32):
                for i in range(15, 16):
                    coord = rff.get(type2chains[f"fg{fg_i}"][0][i]).get_translation()
                    if coord[2] > min_z_angstrom:
                        x_coords.append(coord[0] / 10)
                        y_coords.append(coord[1] / 10)
                        z_coords.append(coord[2] / 10)
    return x_coords, y_coords, z_coords


    

In [5]:
x_coords, y_coords, z_coords = load_data("/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_200uM_10ns/", 0)
# x_coords, y_coords, z_coords = load_data("/cs/usr/roi.eliasian/LabFolder/NpcTransportExperiment/HS-AFM-Dataset/dataset/mini_0uM_10ns/")
with open("sr_microscopy_data.pickle", "wb") as f:
    pickle.dump([x_coords, y_coords, z_coords], f)